In [ ]:
#if u haven't install the google-play scrapper library u can install it by this code:
!pip install google-play-scraper

In [ ]:
from google_play_scraper import Sort, reviews_all, reviews, app
import pandas as pd
import numpy as np

In [ ]:
result, continuation_token = reviews(
    'org.detikcom.rss',
    lang='id',                
    country='id',             
    sort=Sort.MOST_RELEVANT,  
    count=10000,               
    filter_score_with=None    
)

In [ ]:
data = pd.DataFrame(np.array(result),columns=['review'])

data = data.join(pd.DataFrame(data.pop('review').tolist()))

data

In [ ]:
scrappeddata1 = data[['content','score','at']]
data = scrappeddata1.sort_values(by='at', ascending=False) #Sort by Newest, change to True if you want to sort by Oldest.
data.head()

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

positive = sample_reviews[sample_reviews['score'] > 3]
neutral = sample_reviews[sample_reviews['score'] == 3]
negative = sample_reviews[sample_reviews['score'] < 3]
scores = ('Positive','Neutral', 'Negative')
y_pos = np.arange(len(scores))
numscores = [len(positive), len(neutral),len(negative)]
 
plt.bar(scores, numscores, align='center', alpha=0.5)
plt.xticks(y_pos, scores)
plt.ylabel('Count')
plt.title('Number of Reviews with its Sentiment')
plt.show()

In [ ]:
del(data['at'])

In [ ]:
del(data['score'])

## Pelabelan menggunakan metode VADER sentiment

In [ ]:
# Get sentiment score for each review
vader_sentiment = SentimentIntensityAnalyzer()
data['scores_VADER'] = data['content'].apply(lambda s: vader_sentiment.polarity_scores(s)['compound'])
# Predict sentiment label for each review
data['pred_VADER'] = data['scores_VADER'].apply(lambda x: 1 if x >=0 else 0)
data

In [ ]:
 Check the counts of labels
data['pred_VADER'].value_counts()

In [ ]:
# random split train and test data
index = data.index
data['random_number'] = np.random.randn(len(index))
train = data[data['random_number'] <= 0.8]
test = data[data['random_number'] > 0.8]

In [ ]:
# count vectorizer:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['content'])
test_matrix = vectorizer.transform(test['content'])

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
X_train = train_matrix
X_test = test_matrix
y_train = train['pred_VADER']
y_test = test['pred_VADER']

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
predictions = lr.predict(X_test)

In [ ]:
# find accuracy, precision, recall:
from sklearn.metrics import confusion_matrix,classification_report
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)

In [ ]:
print(classification_report(predictions,y_test))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize=(8,5))
sns.heatmap(confusion_matrix(predictions, y_test), annot=True, fmt=".0f", ax=ax)
plt.xlabel("y_head")
plt.ylabel("y_true")
plt.show()

## Processing

In [ ]:
import re
#case folding
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr
a['lwr'] = a['content'].apply(clean_lower)

#Remove Puncutuation
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text  
a['clean_punct'] = a['lwr'].apply(clean_punct)

#remove whitespace
def _normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")
a['clean_double_ws'] = a['clean_punct'].apply(_normalize_whitespace)

#porterstemmer
!pip install nltk
import nltk 
wn= nltk.WordNetLemmatizer()
def lemmatization(text):
    text = ' '.join(wn.lemmatize(word) for word in text.split() if word in text)
    return text
# Buat kolom tambahan untuk data description yang telah dilemmatization   
a['desc_clean_porterstem'] = a['clean_double_ws'].apply(lemmatization)

#stopword removal
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


#clean stopwords
stopword = set(stopwords.words('english'))
def clean_stopwords(text):
    text = ' '.join(word for word in text.split() if word not in stopword) # hapus stopword dari kolom deskripsi
    return text
# Buat kolom tambahan untuk data description yang telah distopwordsremoval   
a['clean_sw'] = a['desc_clean_porterstem'].apply(clean_stopwords)
add = pd.DataFrame(a['clean_sw'])
a['add_swr']= add.replace(to_replace =['donk','sll'],  
                            value ="", regex= True) 

## Pembobotan tf-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#Perhitungan TF
def calc_TF(document):
    # Counts the number of times the word appears in review
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    # Computes tf for each word
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

sample_reviews["TF_dict"] = sample_reviews['text_stem'].apply(calc_TF)

sample_reviews["TF_dict"].head()

In [ ]:
# Check TF result (Hasil perhitungan TF)
index = 15

print('%20s' % "term", "\t", "TF\n")
for key in sample_reviews["TF_dict"][index]:
    print('%20s' % key, "\t", sample_reviews["TF_dict"][index][key])

In [ ]:
def calc_DF(tfDict):
    count_DF = {}
    # Run through each document's tf dictionary and increment countDict's (term, doc) pair
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF

DF = calc_DF(sample_reviews["TF_dict"])
DF

In [ ]:
import numpy as np
n_document = len(sample_reviews)

def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict
  
#Stores the idf dictionary
IDF = calc_IDF(n_document, DF)
IDF

In [ ]:
#calc TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    #For each word in the review, we multiply its tf and its idf.
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#Stores the TF-IDF Series
sample_reviews["TF-IDF_dict"] = sample_reviews["TF_dict"].apply(calc_TF_IDF)

In [ ]:
# Check TF-IDF result
index = 15

print('%20s' % "term", "\t", '%10s' % "TF", "\t", '%20s' % "TF-IDF\n")
for key in sample_reviews["TF-IDF_dict"][index]:
    print('%20s' % key, "\t", sample_reviews["TF_dict"][index][key] ,"\t" , sample_reviews["TF-IDF_dict"][index][key])


In [ ]:
# sort descending by value for DF dictionary 
sorted_DF = sorted(DF.items(), key=lambda kv: kv[1], reverse=True)[:50]

# Create a list of unique words from sorted dictionay `sorted_DF`
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)

    # For each unique word, if it is in the review, store its TF-IDF value.
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

sample_reviews["TF_IDF_Vec"] = sample_reviews["TF-IDF_dict"].apply(calc_TF_IDF_Vec)

print("print first row matrix TF_IDF_Vec Series\n")
print(sample_reviews["TF_IDF_Vec"][0])

print("\nmatrix size : ", len(sample_reviews["TF_IDF_Vec"][0]))

In [ ]:
# Convert Series to List
TF_IDF_Vec_List = np.array(sample_reviews["TF_IDF_Vec"].to_list())

# Sum element vector in axis=0 
sums = TF_IDF_Vec_List.sum(axis=0)

data = []

for col, term in enumerate(unique_term):
    data.append((term, sums[col]))
    
ranking = pd.DataFrame(data, columns=['term', 'rank'])
ranking.sort_values('rank', ascending=False)

In [ ]:
#ranking.head(11).plot.bar(x='term', y='rank', rot=0,figsize=(40,20));
plt.figure(figsize=(9,6))
plt.bar(x=ranking['term'],
 
        height=ranking['rank'],
       color='midnightblue')
plt.xticks(rotation=90)
plt.yticks(fontsize = 13)
plt.title('TF_IDF_Vec_List')

## Wordcloud

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
!pip install plotly
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode()
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
list_colors = ["#17C37B", "#F92969", "#FACA0C"]
list_titles = ["Positive Sentiment", "Negative Sentiment", "Neutral Sentiment"]

fig, axes = plt.subplots(1, 3, figsize=(30, 20), sharey=True, dpi=160)

for i, ax in enumerate (axes.flatten()):
    wc = WordCloud(
        width=800,
        height=800,
        max_words=50,
        min_font_size=10,
        background_color="white",
        colormap="tab10",
        color_func=lambda *args, **kwargs: list_colors[i],
        stopwords=None,
        prefer_horizontal=1.0
    )
    fig.add_subplot(ax)
    wc.generate(sample_reviews['text_number'][i])
    plt.gca().imshow(wc)
    plt.gca().set_title(list_titles[i], fontdict=dict(size=16))
    plt.gca().axis("off")
    
plt.axis("off")
plt.subplots_adjust(hspace=0.6)
plt.show()

In [ ]:
#filter the df to one candidate, and create a list of responses from them
text = sample_reviews['content'].tolist() 

# join the list and lowercase all the words
text = ' '.join(text).lower()

# change the value to black
def black_color_func(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl(0,100%, 1%)")
#create the wordcloud object
wordcloud = WordCloud(stopwords = STOPWORDS,background_color="white", width=3000, height=2000, max_words=500,
                      collocations=True).generate(text)
# set the word color to black
wordcloud.recolor(color_func = black_color_func)
#plot the wordcloud object
plt.imshow(wordcloud, interpolation='bilInear')
plt.axis('off')
plt.show()

## Classification with MaxEnt

In [ ]:
import collections
import itertools
import nltk
nltk.download('movie_reviews')
import nltk.classify.util, nltk.metrics
from nltk.classify import MaxentClassifier
from nltk.metrics import precision, recall, f_measure
from nltk.corpus import movie_reviews

def word_feats(words):
	return dict([(word, True) for word in words])

negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

negcutoff = int(len(negfeats)*3/4)
poscutoff = int(len(posfeats)*3/4)

trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]

print('train on %d instances, test on %d instances - Maximum Entropy' % (len(trainfeats), len(testfeats)))

classifier = MaxentClassifier.train(trainfeats, 'GIS', trace=0, encoding=None, labels=None, gaussian_prior_sigma=0, max_iter = 1)

refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(testfeats):
	refsets[label].add(i)
	observed = classifier.classify(feats)
	testsets[observed].add(i)
 
accuracy = nltk.classify.util.accuracy(classifier, testfeats)
pos_precision = precision(refsets['pos'], testsets['pos'])
pos_recall = recall(refsets['pos'], testsets['pos'])
pos_fmeasure = f_measure(refsets['pos'], testsets['pos'])
neg_precision = precision(refsets['neg'], testsets['neg'])
neg_recall = recall(refsets['neg'], testsets['neg'])
neg_fmeasure =  f_measure(refsets['neg'], testsets['neg'])
		
print('')
print ('---------------------------------------')
print ('          Maximum Entropy              ')
print ('---------------------------------------')
print ('accuracy:', accuracy)
print ('precision', (pos_precision + neg_precision) / 2)
print ('recall', (pos_recall + neg_recall) / 2)
print ('f-measure', (pos_fmeasure + neg_fmeasure) / 2)